In [1]:
import torch
from torch import nn, optim
import segmentation_models_pytorch as smp


class Unet(nn.Module):
    def __init__(self, num_classes,encoder,pre_weight):
        super().__init__()
        self.model = smp.Unet( classes = num_classes,
                              encoder_name=encoder,
                              encoder_weights=pre_weight,
                              in_channels=3)
    
    def forward(self, x):
        y = self.model(x)
        encoder_weights = "imagenet"
        return y

C:\Users\deepBlue\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 라벨 순서에 맞게 모델 적제

labels = ['Breakage_3', 'Crushed_2', 'Scratch_0', 'Seperated_1']
models = []

n_classes = 2
device = 'cuda' if torch.cuda.is_available() else 'cpu'

for label in labels:
    model_path = f'models/[DAMAGE][{label}]Unet.pt'

    model = Unet(encoder='resnet34', pre_weight='imagenet', num_classes=n_classes).to(device)
    model.model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    model.eval()

    models.append(model)

print('Loaded pretrained models!')

Loaded pretrained models!


In [5]:
import os

# 테스트 이미지 경로
path_test = './data/testimg/'
test_img_list  = os.listdir(path_test)
test_img = test_img_list[0]

# 견적서 경로
# estimate_path = './1.Training/1.원천데이터/TS_99.붙임_견적서/'
# estimate_list = os.listdir(estimate_path)

In [6]:
test_img

'0000008_as-0036229.jpg'

In [24]:
## 테스트 이미지에 id와 car_name 값 입력
import pandas as pd

max_id = 18246
test_car_name = ['']
repair_cost = {
    'Breakage_3':[],
    'Crushed_2':[],
    'Scratch_0':[],
    'Seperated_1':[],    
    'id':[],
    'car_name':[],
    }

repair_cost['id'].append(max_id)
repair_cost['car_name'].append('올뉴모닝')
# repair_cost['id'].append(max_id)
# for j in range(len(estimate_list)):
#     if test_img.split('_')[1].split('.')[0] == estimate_list[j].split('.')[0]:            
#         df = pd.read_excel(estimate_path+estimate_list[j])
#         repair_cost['car_name'].append(df[df['Unnamed: 7']=='차량명칭']['Unnamed: 8'].to_list()[0].split('(')[0])

In [25]:
repair_cost

{'Breakage_3': [],
 'Crushed_2': [],
 'Scratch_0': [],
 'Seperated_1': [],
 'id': [18246],
 'car_name': ['올뉴모닝']}

In [26]:
import torch
import cv2
import matplotlib.pyplot as plt
import re

for img_list in range(1):
    img = cv2.imread(path_test+test_img_list[img_list])    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (256, 256))

    img_input = img/255
    img_input = img_input.transpose([2, 0, 1])
    img_input = torch.tensor(img_input).float().to(device)
    img_input = img_input.unsqueeze(0)
    
    outputs = []
    for i, model in enumerate(models):
        output = model(img_input)

        img_output = torch.argmax(output, dim=1).detach().cpu().numpy()
        img_output = img_output.transpose([1, 2, 0])

        outputs.append(img_output)

    for i, label in enumerate(labels):
        repair_cost[label].append(outputs[i].sum())


# 모든 masking 값이 1로 되어있으므로 모든 값을 더한 값이 면적이 된다
# 그러므로 sum()으로 합쳐주기만 하면 면적을 구할 수 있다.

# output = 이미지를 모델로 훈련한 결과
# outputs = 4개 모델의 결과를 전부 합친 것 

# repair_cost[label] dict에 output의 값을 전부 합친 값을 입력


# import numpy as np
# test = np.array(outputs).flatten()
# print([x for x in test if x!=0])

In [27]:
repair_cost

{'Breakage_3': [0],
 'Crushed_2': [0],
 'Scratch_0': [0],
 'Seperated_1': [678],
 'id': [18246],
 'car_name': ['올뉴모닝']}

In [28]:
import joblib
model_path = './' + 'models/' + 'test_XGB03' +'.model'
new_model = joblib.load(model_path)

In [29]:
df_rp = pd.DataFrame(repair_cost) 
df_rp

,Breakage_3,Crushed_2,Scratch_0,Seperated_1,id,car_name
0,0,0,0,678,18246,올뉴모닝


In [30]:
df_rp['car_name'][0]

'올뉴모닝'

In [31]:
label_info = pd.read_csv('./data/label_data.csv')
for i in range(len(label_info['car_name'])):
    # print(label_info['car_name'][i])
    # print(label_info['labeled'][i])
    # print(df_rp['car_name'][0] == label_info['car_name'][i])
    if df_rp['car_name'][0] == label_info['car_name'][i]:
        df_rp = df_rp.replace(label_info['car_name'][i],label_info['labeled'][i])

In [32]:
label_info = pd.read_csv('./data/label_data.csv')
for i in range(len(label_info['car_name'])):
    if df_rp['car_name'][0] == label_info['car_name'][i]:
        df_rp = df_rp.replace(label_info['car_name'][i],label_info['labeled'][i])
df_rp

,Breakage_3,Crushed_2,Scratch_0,Seperated_1,id,car_name
0,0,0,0,678,18246,217


In [33]:
df_rp.iloc[0].values.reshape(1,6)

array([[    0,     0,     0,   678, 18246,   217]], dtype=int64)

In [34]:
new_model.predict(df_rp.iloc[0].values.reshape(1,6))

array([405082.6], dtype=float32)